In [10]:
import scrapy
import re
import json
import logging
import pandas as pd
import selenium
import time
import sys
import linecache
import selenium.webdriver as webdriver
import selenium.webdriver.support.ui as ui
import pymongo
import datetime
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from scrapy.crawler import CrawlerProcess
from time import sleep
from selenium.webdriver.common.keys import Keys
from time import sleep 
from pymongo import MongoClient 
from selenium.webdriver.common.by import By

# SOGO Scraping

In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
 
def scrape_url(url,collection):
    print(collection)
    # Connect to mongodb
    try: 
        conn = MongoClient() 
        print("Connected successfully!!!") 
    except:   
        print("Could not connect to MongoDB") 
    # database
    db = conn.sogo
    # Created or Switched to collection names: pakaian_wanita 
    collection = db[collection]
    
    # Load WebDriver and navigate to the page url.
    # This will open a browser window.
    driver = webdriver.Firefox()
    driver.get(url)

    # First scroll to the end of the page
    # Click view more
    lastHeight = driver.execute_script("return document.body.scrollHeight")
    error = 0
    while error<10:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        newHeight = driver.execute_script("return document.body.scrollHeight")
        if newHeight == lastHeight:
            # Click view more
            try:
                submit_button = driver.find_elements_by_xpath('//*[@id="wrapper"]/section/article[2]/div[3]/div[1]/a')[0]
                submit_button.click()
                print("view more")
                view_more = False
                error=0
            except Exception as e:
                print(str(e))
                time.sleep(10)
                error = error+1

        lastHeight = newHeight

    # Parsing the result
    print('start parsing')
    path = driver.find_elements_by_xpath('//*[@id="wrapper"]/section/article[2]/div[2]/div/div')

    result = pd.DataFrame()
    for x in range(len(path)):
        discounted_price = re.sub("\.","",re.sub("Rp. ","",path[x].find_elements_by_xpath('div/div[2]/div/div[2]/span')[0].text))
        try:
            origin_price = re.sub("\.","",re.sub("Rp. ","",path[x].find_elements_by_xpath('div/div[2]/div/div[2]/del')[0].text))
        except:
            origin_price = discounted_price
        id_product = path[x].find_elements_by_xpath('div/div[2]/div/ul/li')[0].get_attribute('data-id')
        name = path[x].find_elements_by_xpath('div/div[2]/div/ul/li')[0].get_attribute('data-title')
        brand = path[x].find_elements_by_xpath('div/div[2]/div/ul/li')[0].get_attribute('data-brandname')
        image = re.sub(".jpg.*", ".jpg","https:" + path[x].find_elements_by_xpath('div/div[2]/div/ul/li')[0].get_attribute('data-img-src'))
        category = path[x].find_elements_by_xpath('div/div[2]/div/ul/li')[0].get_attribute('data-categorytree')
        code = path[x].find_elements_by_xpath('div/div[2]/div/ul/li')[0].get_attribute('data-code')
        style = path[x].find_elements_by_xpath('div/div[2]/div/ul/li')[0].get_attribute('data-style')
        new_result = pd.DataFrame([{'id_product':id_product,'discounted_price':discounted_price, 'origin_price':origin_price, 'name':name, 'brand':brand, 'image':image,'category':category,'code':code,'style':style}])
        result=result.append(new_result)
        
    # save to mongodb
    try:
        collection.insert_many(result.to_dict('records'))
    except:
        PrintException()
        pass

In [11]:
scrape_url("https://www.mapemall.com/forkids/girls","girls")
scrape_url("https://www.mapemall.com/forkids/boys","boys")
scrape_url("https://www.mapemall.com/shop/SOGO/women/clothing","women_clothing")
scrape_url("https://www.mapemall.com/shop/SOGO/women/shoes","women_shoes")
scrape_url("https://www.mapemall.com/shop/SOGO/men/clothing","men_clothing")
scrape_url("https://www.mapemall.com/shop/SOGO/men/shoes","men_shoes")

girls
Connected successfully!!!
view more
view more
view more
view more
view more
Message: Element <a class="btn btn-black-border"> could not be scrolled into view

Message: Element <a class="btn btn-black-border"> could not be scrolled into view

Message: Element <a class="btn btn-black-border"> could not be scrolled into view

Message: Element <a class="btn btn-black-border"> could not be scrolled into view

Message: Element <a class="btn btn-black-border"> could not be scrolled into view

Message: Element <a class="btn btn-black-border"> could not be scrolled into view

Message: Element <a class="btn btn-black-border"> could not be scrolled into view

Message: Element <a class="btn btn-black-border"> could not be scrolled into view

Message: Element <a class="btn btn-black-border"> could not be scrolled into view

Message: Element <a class="btn btn-black-border"> could not be scrolled into view

start parsing
boys
Connected successfully!!!
view more
view more
view more
view more
Mes